In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key
openai_api_key = os.environ.get('openai_api_key')

if not openai_api_key:
    print("⚠️  openai_api_key not found in .env file")
else:
    print("✓ OpenAI API key loaded from .env file")
    
# Get Gemini API key
gemini_api_key = os.environ.get('gemini_api_key')

if not gemini_api_key:
    print("⚠️  gemini_api_key not found in .env file")
    print("Please add: gemini_api_key=your-api-key to .env file")
else:
    print("✓ Gemini API key loaded from .env file")
    
# Create OpenAI client
client = OpenAI(api_key=openai_api_key)

# Configure Gemini
genai.configure(api_key=gemini_api_key)
gemini_model = genai.GenerativeModel('gemini-2.0-flash-exp')


In [ ]:
# This is a test to make sure it is working
user_prompt = "Who am I talking to?"

print(f"\n📤 Sending to openai: {user_prompt}")
print("-" * 60)

# Send to openai
response = client.chat.completions.create(
    model="gpt-4",  # or "gpt-3.5-turbo" for faster/cheaper responses
    messages=[
        {"role": "user", "content": user_prompt}
    ]
)

# Get the response
ai_response = response.choices[0].message.content

print("\n📥 openai Response:")
print(ai_response)


## Advanced: Gradio Chatbot with Conversation History

This creates a chatbot interface that remembers your conversation


In [ ]:
import gradio as gr

def dual_ai_response(message):
    """
    Get OpenAI response and Gemini evaluation
    """
    try:
        # Get OpenAI response
        openai_response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": message}]
        )
        openai_answer = openai_response.choices[0].message.content
        
        # Get Gemini evaluation
        gemini_prompt = f"""Evaluate this AI response:

Question: {message}
Response: {openai_answer}

Rate it (1-10) and explain why. Is it accurate and helpful?"""
        
        gemini_response = gemini_model.generate_content(gemini_prompt)
        gemini_evaluation = gemini_response.text
        
        return openai_answer, gemini_evaluation
        
    except Exception as e:
        return f"Error: {str(e)}", "Unable to evaluate"

# Create custom interface with two output boxes
with gr.Blocks() as demo:
    gr.Markdown("# AI Response with Gemini Quality Check")
    gr.Markdown("Ask a question to OpenAI GPT-4, and Gemini will evaluate the response quality!")
    
    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(
                label="Your Question", 
                lines=3,
                placeholder="Ask anything..."
            )
            submit_btn = gr.Button("🚀 Ask & Evaluate", variant="primary")
        
    with gr.Row():
        with gr.Column():
            openai_output = gr.Textbox(
                label="🤖 OpenAI GPT-4 Response", 
                lines=15,
                show_copy_button=True
            )
        with gr.Column():
            gemini_output = gr.Textbox(
                label="✨ Gemini Quality Evaluation", 
                lines=15,
                show_copy_button=True
            )
    
    # Example questions
    gr.Examples(
        examples=[
            "Explain quantum computing in simple terms",
            "Write a haiku about artificial intelligence",
            "What is the difference between machine learning and deep learning?",
            "How does photosynthesis work?"
        ],
        inputs=user_input
    )
    
    submit_btn.click(
        fn=dual_ai_response,
        inputs=user_input,
        outputs=[openai_output, gemini_output]
    )

# Launch with public sharing enabled
demo.launch(share=True)


In [ ]:
# Close the Gradio interface
demo.close()